In [19]:
# Importamos las librerias que vamos a usar
import requests
from urllib.error import HTTPError
from urllib.error import URLError
from bs4 import BeautifulSoup
from bs4.dammit import EncodingDetector
import pandas as pd
import numpy as np
import datetime

# Crear listas para guardar los datos
marcas = []
gas95s = []
gas98s = []
gasoils = []
marcas = []
direcciones = []
poblaciones = []
horarios = []

# Definimos la funcion que busca y recupera los enlaces de las gasolineras de Barcelona
def get_comunity_links(url):
    comu_links = []
    provi_links = []
    try:
        page = requests.get(url)
    except HTTPError as e:
        print(e)
    except URLError:
        print("Server down or incorrect domain")
    else:
        res = BeautifulSoup(page.content,"html.parser")
        for link in res.find_all('a'):
            if link.has_attr('href'):
                if '/c/precio-gasoil-calefaccion-' in link.attrs['href']:
                    comu_links.append(link.attrs['href'])
                if '/p/precio-gasoil-calefaccion-' in link.attrs['href']:
                    provi_links.append(link.attrs['href'])
    return comu_links, provi_links

def get_provincia_links(comu_links,provi_links):
    for comu_link in comu_links:
        try:
            html = ("https://www.clickgasoil.com" + comu_link)
            page = requests.get(html)
        except HTTPError as e:
            print(e)
        except URLError:
            print("Server down or incorrect domain")
        else:
            #res = BeautifulSoup(open(page, 'r'),"html.parser",from_encoding="iso-8859-1")
            res = BeautifulSoup(page.content,"html.parser", from_encoding="latin1")
        for link in res.find_all('a'):
            if link.has_attr('href'):
                if '/p/precio-gasoil-calefaccion-' in link.attrs['href']:
                    provi_links.append(link.attrs['href'])
                if '/p/precio-de-gasoil-calefaccion-' in link.attrs['href']:
                    provi_links.append(link.attrs['href'])
    return provi_links

def get_city_links(provi_links):
    city_links = []
    for provi_link in provi_links:
        try:
            html = ("https://www.clickgasoil.com" + provi_link)
            page = requests.get(html)
        except HTTPError as e:
            print(e)
        except URLError:
            print("Server down or incorrect domain")
        else:
            #soup = BeautifulSoup(open(page, 'r'),"html.parser",from_encoding="iso-8859-1")
            soup = BeautifulSoup(page.content,"html.parser", from_encoding="latin1")
        for link in soup.find_all('a'):
            if link.has_attr('href'):
                if '/m/precio-gasoil-calefaccion-' in link.attrs['href']:
                    city_links.append(link.attrs['href'])
                    #print(link.attrs['href'])
                if '/m/precio-de-gasoil-calefaccion-' in link.attrs['href']:
                    city_links.append(link.attrs['href'])
                    #print(link.attrs['href'])
    return city_links

def get_station_links(city_links):
    station_links = []
    for city_link in city_links:
        try:
            html = ("https://www.clickgasoil.com" + city_link)
            page = requests.get(html)
        except HTTPError as e:
            print(e)
        except URLError:
            print("Server down or incorrect domain")
        else:
            # Estructuramos el contenido de la web obtenido para ser consultado facilmente
            estacion = BeautifulSoup(page.content,"html.parser")

            #Buscamos los links:
            for link in estacion.find_all('a'):
                if link.has_attr('href'):
                    if '/g/' in link.attrs['href']:
                        station_links.append(link.attrs['href'])
    return station_links

# Definimos la funcion que busca y recupera todos los datos referente a cada gasolinera
def get_info(station):
    print (station)
    try:
        html = ("https://www.clickgasoil.com" + station)
        page = requests.get(html)
    except HTTPError as e:
        print(e)
    except URLError:
        print("Server down or incorrect domain")
    else:
        # Estructuramos el contenido de la web obtenido para ser consultado facilmente
        soup = BeautifulSoup(page.content,'html.parser')
        
        # Buscamos la información general de cada gasolinera: Marca, Direcion, Poblacion, Horario
        info_gasolinera = soup.find("div", {"class":"column small-4 datos_gasolinera"})
        
        # Buscamos ahora las etiquetas div que hemos detectado, son las que engloban la información que buscamos
        div_lista = info_gasolinera.find_all('div')
        
        # Recorremos la información y guardamos los datos que nos interesan
        for i,dato in enumerate(div_lista):
            temp = dato.text.split(":")
            if (i==0): 
                marca = temp[1]
            if (i==1):
                direccion = temp[1]
            if (i==2):
                poblacion = temp[1]
            if (i==3):
                horario = temp[1] + temp[2]
                
        # Buscamos la tabla que contiene los precios de los combustibles a fecha de la consulta
        table = soup.find("table", { "class" : "small-12" })
        
        # Buscamos todas las filas de esta tabla y las guardamos en una lista
        td_list = table.find_all('td')
        
        # Recorremos la información y guardamos los datos que nos interesan
        for i,element in enumerate(td_list):
            if(i == 1):
                precio_gasoil = td_list[1].text[:-1]
                gasoil = precio_gasoil
            if (i == 7):
                precio_gas95 = td_list[7].text[:-1]
                gas95 = precio_gas95
            if (i == 9):
                precio_gas98 = td_list[9].text[:-1]
                gas98 = precio_gas98
            
        # Devolvemos los datos
    return marca,gasoil,gas95,gas98,direccion,poblacion,horario

def precios_gasolineras(station_links):
    # Para cada estacion, vamos a su pagina donde obtenemos todos sus datos y los precios del combustible del dia
    for station in station_links:
        marca,gasoil,gas95,gas98,direccion,poblacion,horario = get_info(station)

        # Guardamos los datos en listas que nos permitiran luego crear un dataframe que exportaremos.
        marcas.append(marca)
        gasoils.append(gasoil)
        gas95s.append(gas95)
        gas98s.append(gas98)
        direcciones.append(direccion)
        poblaciones.append(poblacion)
        horarios.append(horario)

    # Creamos un dataframe con todos los datos obtenidos
    df = pd.DataFrame(
        {'Marca':marcas,
         'Gasolina 95':gas95s,
         'Gasolina 98':gas98s,
         'Gasóleo A':gasoils,
         'Direccion':direcciones,
         'Poblacion':poblaciones,
         'Horario':horarios,
         'Fecha': datetime.datetime.now()})
    return df



# Parseando la web ClickGasoil que nos ofrece los precios de Gasóleo A, Gasolina 95 y Gasolina 98 de cada gasolinera de España
url = "https://www.clickgasoil.com/c/precio-gasoil-calefaccion"
comu_links, provi_links = get_comunity_links(url)
print ("Ya tenemos las comunidades, son:" + " " + str(len(comu_links + provi_links)))
       
provi_links = get_provincia_links(comu_links,provi_links)
print ("Ya tenemos las provincias, son:" + " " + str(len(provi_links)))

bcn_provincia = ""
for station in provi_links:
    if "barcelona" in station:
        bcn_provincia = station

city_links = get_city_links(bcn_provincia)
print ("Ya tenemos las ciudades, son:" + " " + str(len(city_links)))
station_links = get_station_links(city_links)
print ("Ya tenemos las estaciones de servicio! y son: " + " " + str(len(station_links)))


Ya tenemos las comunidades, son: 19
Ya tenemos las provincias, son: 52
['/p/precio-gasoil-calefaccion-en-asturias', '/p/precio-gasoil-calefaccion-en-illes-balears', '/p/precio-gasoil-calefaccion-en-cantabria', '/p/precio-gasoil-calefaccion-en-madrid', '/p/precio-gasoil-calefaccion-en-murcia', '/p/precio-gasoil-calefaccion-en-navarra', '/p/precio-gasoil-calefaccion-en-la-rioja', '/p/precio-gasoil-calefaccion-en-ceuta', '/p/precio-gasoil-calefaccion-en-melilla', '/p/precio-gasoil-calefaccion-en-sevilla', '/p/precio-gasoil-calefaccion-en-malaga', '/p/precio-gasoil-calefaccion-en-cordoba', '/p/precio-gasoil-calefaccion-en-cadiz', '/p/precio-gasoil-calefaccion-en-almeria', '/p/precio-gasoil-calefaccion-en-jaen', '/p/precio-gasoil-calefaccion-en-huelva', '/p/precio-gasoil-calefaccion-en-granada', '/p/precio-gasoil-calefaccion-en-zaragoza', '/p/precio-gasoil-calefaccion-en-huesca', '/p/precio-gasoil-calefaccion-en-teruel', '/p/precio-gasoil-calefaccion-en-valladolid', '/p/precio-gasoil-calefa

/m/precio-de-gasoil-calefaccion-santander
/m/precio-de-gasoil-calefaccion-torrelavega
/m/precio-de-gasoil-calefaccion-castro-urdiales
/m/precio-de-gasoil-calefaccion-camargo
/m/precio-de-gasoil-calefaccion-pielagos
/m/precio-de-gasoil-calefaccion-astillero-el
/m/precio-de-gasoil-calefaccion-santa-cruz-de-bezana
/m/precio-de-gasoil-calefaccion-laredo
/m/precio-de-gasoil-calefaccion-santoa
/m/precio-de-gasoil-calefaccion-los-corrales-de-buelna
/m/precio-de-gasoil-calefaccion-reinosa
/m/precio-de-gasoil-calefaccion-santa-maria-de-cayon
/m/precio-de-gasoil-calefaccion-suances
/m/precio-de-gasoil-calefaccion-reocin
/m/precio-de-gasoil-calefaccion-cabezon-de-la-sal
/m/precio-de-gasoil-calefaccion-colindres
/m/precio-de-gasoil-calefaccion-medio-cudeyo
/m/precio-de-gasoil-calefaccion-polanco
/m/precio-de-gasoil-calefaccion-cartes
/m/precio-de-gasoil-calefaccion-marina-de-cudeyo
/m/precio-de-gasoil-calefaccion-entrambasaguas
/m/precio-de-gasoil-calefaccion-miengo
/m/precio-de-gasoil-calefaccion

/m/precio-de-gasoil-calefaccion-murcia
/m/precio-de-gasoil-calefaccion-cartagena
/m/precio-de-gasoil-calefaccion-lorca
/m/precio-de-gasoil-calefaccion-molina-de-segura
/m/precio-de-gasoil-calefaccion-alcantarilla
/m/precio-de-gasoil-calefaccion-cieza
/m/precio-de-gasoil-calefaccion-aguilas
/m/precio-de-gasoil-calefaccion-torre-pacheco
/m/precio-de-gasoil-calefaccion-yecla
/m/precio-de-gasoil-calefaccion-mazarron
/m/precio-de-gasoil-calefaccion-san-javier
/m/precio-de-gasoil-calefaccion-totana
/m/precio-de-gasoil-calefaccion-caravaca-de-la-cruz
/m/precio-de-gasoil-calefaccion-jumilla
/m/precio-de-gasoil-calefaccion-san-pedro-del-pinatar
/m/precio-de-gasoil-calefaccion-las-torres-de-cotillas
/m/precio-de-gasoil-calefaccion-alhama-de-murcia
/m/precio-de-gasoil-calefaccion-union-la
/m/precio-de-gasoil-calefaccion-archena
/m/precio-de-gasoil-calefaccion-mula
/m/precio-de-gasoil-calefaccion-fuente-alamo-de-murcia
/m/precio-de-gasoil-calefaccion-santomera
/m/precio-de-gasoil-calefaccion-ceheg

/m/precio-de-gasoil-calefaccion-logroo
/m/precio-de-gasoil-calefaccion-calahorra
/m/precio-de-gasoil-calefaccion-arnedo
/m/precio-de-gasoil-calefaccion-haro
/m/precio-de-gasoil-calefaccion-alfaro
/m/precio-de-gasoil-calefaccion-lardero
/m/precio-de-gasoil-calefaccion-najera
/m/precio-de-gasoil-calefaccion-villamediana-de-iregua
/m/precio-de-gasoil-calefaccion-santo-domingo-de-la-calzada
/m/precio-de-gasoil-calefaccion-autol
/m/precio-de-gasoil-calefaccion-pradejon
/m/precio-de-gasoil-calefaccion-rincon-de-soto
/m/precio-de-gasoil-calefaccion-albelda-de-iregua
/m/precio-de-gasoil-calefaccion-fuenmayor
/m/precio-de-gasoil-calefaccion-navarrete
/m/precio-de-gasoil-calefaccion-aldeanueva-de-ebro
/m/precio-de-gasoil-calefaccion-alberite
/m/precio-de-gasoil-calefaccion-cervera-del-rio-alhama
/m/precio-de-gasoil-calefaccion-ezcaray
/m/precio-de-gasoil-calefaccion-cenicero
/m/precio-de-gasoil-calefaccion-quel
/m/precio-de-gasoil-calefaccion-murillo-de-rio-leza
/m/precio-de-gasoil-calefaccion-b

/m/precio-de-gasoil-calefaccion-malaga
/m/precio-de-gasoil-calefaccion-marbella
/m/precio-de-gasoil-calefaccion-mijas
/m/precio-de-gasoil-calefaccion-velez-malaga
/m/precio-de-gasoil-calefaccion-fuengirola
/m/precio-de-gasoil-calefaccion-torremolinos
/m/precio-de-gasoil-calefaccion-estepona
/m/precio-de-gasoil-calefaccion-benalmadena
/m/precio-de-gasoil-calefaccion-rincon-de-la-victoria
/m/precio-de-gasoil-calefaccion-antequera
/m/precio-de-gasoil-calefaccion-alhaurin-de-la-torre
/m/precio-de-gasoil-calefaccion-ronda
/m/precio-de-gasoil-calefaccion-cartama
/m/precio-de-gasoil-calefaccion-alhaurin-el-grande
/m/precio-de-gasoil-calefaccion-coin
/m/precio-de-gasoil-calefaccion-nerja
/m/precio-de-gasoil-calefaccion-torrox
/m/precio-de-gasoil-calefaccion-manilva
/m/precio-de-gasoil-calefaccion-alora
/m/precio-de-gasoil-calefaccion-pizarra
/m/precio-de-gasoil-calefaccion-campillos
/m/precio-de-gasoil-calefaccion-archidona
/m/precio-de-gasoil-calefaccion-benahavis
/m/precio-de-gasoil-calefacc

...

/m/precio-de-gasoil-calefaccion-corua-a
/m/precio-de-gasoil-calefaccion-santiago-de-compostela
/m/precio-de-gasoil-calefaccion-ferrol
/m/precio-de-gasoil-calefaccion-naron
/m/precio-de-gasoil-calefaccion-oleiros
/m/precio-de-gasoil-calefaccion-carballo
/m/precio-de-gasoil-calefaccion-arteixo
/m/precio-de-gasoil-calefaccion-ames
/m/precio-de-gasoil-calefaccion-culleredo
/m/precio-de-gasoil-calefaccion-ribeira
/m/precio-de-gasoil-calefaccion-cambre
/m/precio-de-gasoil-calefaccion-boiro
/m/precio-de-gasoil-calefaccion-teo
/m/precio-de-gasoil-calefaccion-sada
/m/precio-de-gasoil-calefaccion-noia
/m/precio-de-gasoil-calefaccion-fene
/m/precio-de-gasoil-calefaccion-betanzos
/m/precio-de-gasoil-calefaccion-ordes
/m/precio-de-gasoil-calefaccion-laracha-a
/m/precio-de-gasoil-calefaccion-rianxo
/m/precio-de-gasoil-calefaccion-pontes-de-garcia-rodriguez-as
/m/precio-de-gasoil-calefaccion-santa-comba
/m/precio-de-gasoil-calefaccion-pobra-do-caramial-a
/m/precio-de-gasoil-calefaccion-porto-do-son
/

/m/precio-de-gasoil-calefaccion-ourense
/m/precio-de-gasoil-calefaccion-verin
/m/precio-de-gasoil-calefaccion-carballio-o
/m/precio-de-gasoil-calefaccion-barco-de-valdeorras-o
/m/precio-de-gasoil-calefaccion-barbadas
/m/precio-de-gasoil-calefaccion-xinzo-de-limia
/m/precio-de-gasoil-calefaccion-pereiro-de-aguiar-o
/m/precio-de-gasoil-calefaccion-allariz
/m/precio-de-gasoil-calefaccion-celanova
/m/precio-de-gasoil-calefaccion-ribadavia
/m/precio-de-gasoil-calefaccion-san-cibrao-das-vias
/m/precio-de-gasoil-calefaccion-rua-a
/m/precio-de-gasoil-calefaccion-coles
/m/precio-de-gasoil-calefaccion-viana-do-bolo
/m/precio-de-gasoil-calefaccion-maceda
/m/precio-de-gasoil-calefaccion-maside
/m/precio-de-gasoil-calefaccion-cartelle
/m/precio-de-gasoil-calefaccion-monterrei
/m/precio-de-gasoil-calefaccion-boboras
/m/precio-de-gasoil-calefaccion-toen
/m/precio-de-gasoil-calefaccion-san-cristovo-de-cea
/m/precio-de-gasoil-calefaccion-amoeiro
/m/precio-de-gasoil-calefaccion-pobra-de-trives-a
/m/prec

/m/precio-de-gasoil-calefaccion-deba
/m/precio-de-gasoil-calefaccion-irun
/m/precio-de-gasoil-calefaccion-errenteria
/m/precio-de-gasoil-calefaccion-eibar
/m/precio-de-gasoil-calefaccion-zarautz
/m/precio-de-gasoil-calefaccion-arrasatemondragon
/m/precio-de-gasoil-calefaccion-hernani
/m/precio-de-gasoil-calefaccion-tolosa
/m/precio-de-gasoil-calefaccion-lasarte-oria
/m/precio-de-gasoil-calefaccion-hondarribia
/m/precio-de-gasoil-calefaccion-pasaia
/m/precio-de-gasoil-calefaccion-bergara
/m/precio-de-gasoil-calefaccion-azpeitia
/m/precio-de-gasoil-calefaccion-andoain
/m/precio-de-gasoil-calefaccion-beasain
/m/precio-de-gasoil-calefaccion-elgoibar
/m/precio-de-gasoil-calefaccion-azkoitia
/m/precio-de-gasoil-calefaccion-oati
/m/precio-de-gasoil-calefaccion-oiartzun
/m/precio-de-gasoil-calefaccion-zumarraga
/m/precio-de-gasoil-calefaccion-ordizia
/m/precio-de-gasoil-calefaccion-zumaia
/m/precio-de-gasoil-calefaccion-legazpi
/m/precio-de-gasoil-calefaccion-aretxabaleta
/m/precio-de-gasoil-c

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHA

Ya tenemos las estaciones de servicio! y son:  764


In [21]:
# Obtenemos los datos de cada estacion y los guardamos en un dataset
# Consulta 1 estacion cada 5 seg. 764 consultas : 64 min aproximadamente
df = precios_gasolineras(station_links) 
df.head(10)

['/g/precios-combustible-gasolinera-petromiralles-calle-port-de-haifa-sn-en-el-municipio-barcelona', '/g/precios-combustible-gasolinera-tortuga-calle-major-26-en-el-municipio-barcelona', '/g/precios-combustible-gasolinera-petronieves--calle-lletra-b-de-la-zona-franca-13-en-el-municipio-barcelona', '/g/precios-combustible-gasolinera-cepsa-calle-caracas-24-en-el-municipio-barcelona', '/g/precios-combustible-gasolinera-confort-auto-carrer-numero-tres-50-en-el-municipio-barcelona', '/g/precios-combustible-gasolinera-repsol-calle-6---sector--c----zona-franca-sn-en-el-municipio-barcelona', '/g/precios-combustible-gasolinera-bp--calle-joan-guell-207-en-el-municipio-barcelona', '/g/precios-combustible-gasolinera-galp-paseo-p-valle-de-hebron-72-en-el-municipio-barcelona', '/g/precios-combustible-gasolinera-galp-calle-tajo-28-en-el-municipio-barcelona', '/g/precios-combustible-gasolinera-cepsa-avenida-clot-181-en-el-municipio-barcelona', '/g/precios-combustible-gasolinera-repsol-avenida-meridian

/g/precios-combustible-gasolinera-tortuga-calle-major-26-en-el-municipio-barcelona
/g/precios-combustible-gasolinera-petronieves--calle-lletra-b-de-la-zona-franca-13-en-el-municipio-barcelona
/g/precios-combustible-gasolinera-cepsa-calle-caracas-24-en-el-municipio-barcelona
/g/precios-combustible-gasolinera-confort-auto-carrer-numero-tres-50-en-el-municipio-barcelona
/g/precios-combustible-gasolinera-repsol-calle-6---sector--c----zona-franca-sn-en-el-municipio-barcelona
/g/precios-combustible-gasolinera-bp--calle-joan-guell-207-en-el-municipio-barcelona
/g/precios-combustible-gasolinera-galp-paseo-p-valle-de-hebron-72-en-el-municipio-barcelona
/g/precios-combustible-gasolinera-galp-calle-tajo-28-en-el-municipio-barcelona
/g/precios-combustible-gasolinera-cepsa-avenida-clot-181-en-el-municipio-barcelona
/g/precios-combustible-gasolinera-repsol-avenida-meridiana-267-en-el-municipio-barcelona
/g/precios-combustible-gasolinera-repsol-avenida-virgen-de-montserrat-87-en-el-municipio-barcelon

/g/precios-combustible-gasolinera-tamoil-calle-pujades-309-en-el-municipio-barcelona
/g/precios-combustible-gasolinera-tamoil-avenida-fabregada-55-en-el-municipio-hospitalet-de-llobregat-l
/g/precios-combustible-gasolinera-repsol-calle-santa-eulalia-27-en-el-municipio-hospitalet-de-llobregat-l
/g/precios-combustible-gasolinera-galp-autovia-castelldefels-km.-27-en-el-municipio-hospitalet-de-llobregat-l
/g/precios-combustible-gasolinera-galp-autovia-castelldefels-km.-27-en-el-municipio-hospitalet-de-llobregat-l
/g/precios-combustible-gasolinera-petro-g24-calle-calle-cobalt-190-en-el-municipio-hospitalet-de-llobregat-l
/g/precios-combustible-gasolinera-bp-calle-renclusa-36-en-el-municipio-hospitalet-de-llobregat-l
/g/precios-combustible-gasolinera-portuarios-servicios-calle-pablo-iglesias-65-en-el-municipio-hospitalet-de-llobregat-l
/g/precios-combustible-gasolinera-naturgy-calle-botanica-114-en-el-municipio-hospitalet-de-llobregat-l
/g/precios-combustible-gasolinera-ecobenz-avinguda-joan

/g/precios-combustible-gasolinera-galp-calle-domenech-i-montaner-46-en-el-municipio-sabadell
/g/precios-combustible-gasolinera-repsol-butano-carretera-n-150-km.-km-15-en-el-municipio-sabadell
/g/precios-combustible-gasolinera-u.s.sabadell-----gas-oils-rovira--calle-rocafort-40-en-el-municipio-sabadell
/g/precios-combustible-gasolinera-cepsa-calle-molins-de-rei-135137-en-el-municipio-sabadell
/g/precios-combustible-gasolinera-shell-calle-de-rocafort-123-en-el-municipio-sabadell
/g/precios-combustible-gasolinera-cepsa-ronda-sant-pau-de-riu-sec-7-en-el-municipio-sabadell
/g/precios-combustible-gasolinera-repsol-cl-joaquin-blume-1-en-el-municipio-sabadell
/g/precios-combustible-gasolinera-repsol-calle-viladomat-201-en-el-municipio-sabadell
/g/precios-combustible-gasolinera-repsol-avenida-matadepera-247-en-el-municipio-sabadell
/g/precios-combustible-gasolinera-bp-sabadell-cr-terrasa-101-en-el-municipio-sabadell
/g/precios-combustible-gasolinera-repsol-calle-prats-llusanes---esq.-c.-urgel-s

/g/precios-combustible-gasolinera-petrocat-carretera-c-37-km.-9539-en-el-municipio-manresa
/g/precios-combustible-gasolinera-bp-oil-calle-cr.-cardona-8082-en-el-municipio-manresa
/g/precios-combustible-gasolinera-bp-oil-espaa-calle-edison-2-en-el-municipio-manresa
/g/precios-combustible-gasolinera-repsol-calle-carretera-de-vic-225-en-el-municipio-manresa
/g/precios-combustible-gasolinera-repsol-calle-carretera-de-vic-230-en-el-municipio-manresa
/g/precios-combustible-gasolinera-galp-avenida-dels-dolors-5-en-el-municipio-manresa
/g/precios-combustible-gasolinera-cepsa-carretera-c-55-km.-249-en-el-municipio-manresa
/g/precios-combustible-gasolinera-repsol-butano-carretera-de-manresa-al-xup-km.-sn-en-el-municipio-manresa
/g/precios-combustible-gasolinera-repsol-carretera-e-9-km.-147-en-el-municipio-rubi
/g/precios-combustible-gasolinera-repsol-carretera-e-9-km.-149-en-el-municipio-rubi
/g/precios-combustible-gasolinera-repsol-carretera-c-1413-km.-98-en-el-municipio-rubi
/g/precios-combust

...

/g/precios-combustible-gasolinera-truckoil-calle-frana-42-en-el-municipio-franqueses-del-valles-les
/g/precios-combustible-gasolinera-repsol-carretera-c-17-km.-14000-en-el-municipio-parets-del-valles
/g/precios-combustible-gasolinera-cepsa-autovia-c-17-km.-18-en-el-municipio-parets-del-valles
/g/precios-combustible-gasolinera-shell-avinguda-catalunya-111-en-el-municipio-parets-del-valles
/g/precios-combustible-gasolinera-parets-llevantpark-c-tenes-1-en-el-municipio-parets-del-valles
/g/precios-combustible-gasolinera-cepsa-avenida-costa-brava-163-en-el-municipio-malgrat-de-mar
/g/precios-combustible-gasolinera-cepsa-carretera-n-ii-km.-6742-en-el-municipio-malgrat-de-mar
/g/precios-combustible-gasolinera-cepsa-carretera-n-ii-km.-6742-en-el-municipio-malgrat-de-mar
/g/precios-combustible-gasolinera-repsol-cr-n-2-667-en-el-municipio-calella
/g/precios-combustible-gasolinera-repsol-carretera-c-251-km.-55-en-el-municipio-cardedeu
/g/precios-combustible-gasolinera-carrefour-carretera-granolle

/g/precios-combustible-gasolinera-petrocat-carretera-b-502-km.-434-en-el-municipio-argentona
/g/precios-combustible-gasolinera-bp-montgat-calle-ramon-y-cajal-sn-en-el-municipio-montgat
/g/precios-combustible-gasolinera-repsol-carretera-n-ii-km.-597-en-el-municipio-palleja
/g/precios-combustible-gasolinera-cepsa-ronda-santa-eulalia-31-en-el-municipio-palleja
/g/precios-combustible-gasolinera-cepsa-ronda-santa-eulalia-33-en-el-municipio-palleja
/g/precios-combustible-gasolinera-petrocat-directe-carretera-bv-4511-km.-92-en-el-municipio-sant-joan-de-vilatorrada
/g/precios-combustible-gasolinera-tamoil-carretera-c-55-km.-39-en-el-municipio-sant-joan-de-vilatorrada
/g/precios-combustible-gasolinera-shell-poligono-can-messeguer-sn-en-el-municipio-roca-del-valles-la
/g/precios-combustible-gasolinera-repsol-carretera-c-60-km.-87-en-el-municipio-roca-del-valles-la
/g/precios-combustible-gasolinera-repsol-carretera-c-60-km.-87-en-el-municipio-roca-del-valles-la
/g/precios-combustible-gasolinera-r

/g/precios-combustible-gasolinera-meroil-c-rocafort-103-b-4512-km-06-en-el-municipio-artes
/g/precios-combustible-gasolinera-repsol-carretera-bv-5001-sant-adria-la-roca-km.-199-en-el-municipio-vilanova-del-valles
/g/precios-combustible-gasolinera-petrologic-oil-carretera-b510-km.-31-en-el-municipio-dosrius
/g/precios-combustible-gasolinera-galp-calle-sant-galderich-14-en-el-municipio-sant-pol-de-mar
/g/precios-combustible-gasolinera-repsol-barriada-bassacs-sn-en-el-municipio-gironella
/g/precios-combustible-gasolinera-avanti-ctra.-de-cardona-a-berga-b420-km-1.700-en-el-municipio-cardona
/g/precios-combustible-gasolinera-vilalta-carretera-c-1410a-km.-29-en-el-municipio-cardona
/g/precios-combustible-gasolinera-repsol-carretera-n-2-km.-643-en-el-municipio-cabrera-de-mar
/g/precios-combustible-gasolinera-carrefour-carretera-nacional-ii-km.-644-en-el-municipio-cabrera-de-mar
/g/precios-combustible-gasolinera-repsol-b-502-km-1-en-el-municipio-cabrera-de-mar
/g/precios-combustible-gasolinera

/g/precios-combustible-gasolinera-esclatoil-hiper-esclat.-ctra.-c-17-km-55-en-el-municipio-malla
/g/precios-combustible-gasolinera-repsol-carretera-c-25-km.-11152-en-el-municipio-sant-pere-sallavinera
/g/precios-combustible-gasolinera-repsol-carretera-c-25-km.-11152-en-el-municipio-sant-pere-sallavinera
/g/precios-combustible-gasolinera-la-panadella-carretera-nacional-ii-km.-532-en-el-municipio-montmaneu
/g/precios-combustible-gasolinera-la-panadella-carretera-nacional-ii-km.-532-en-el-municipio-montmaneu
/g/precios-combustible-gasolinera-petromiralles-carretera-c-37-km.-48500-en-el-municipio-santa-maria-de-miralles


,Marca,Gasolina 95,Gasolina 98,Gasóleo A,Direccion,Poblacion,Horario,Fecha
0,PETROMIRALLES,1.279,1.392,1.269,"CALLE PORT DE HAIFA, S/N",BARCELONA,L-D 24H,2019-04-14 14:07:58.867659
1,TORTUGA,1.349,,1.259,"CALLE MAJOR, 26",BARCELONA,L-D 24H,2019-04-14 14:07:58.867659
2,PETRONIEVES,1.319,1.419,1.219,"CALLE LLETRA B DE LA ZONA FRANCA, 13",BARCELONA,L-D 24H,2019-04-14 14:07:58.867659
3,CEPSA,1.359,1.474,1.269,"CALLE CARACAS, 24",BARCELONA,L-D 06,2019-04-14 14:07:58.867659
4,CONFORT AUTO,1.339,1.475,1.265,"CARRER NUMERO TRES, 50",BARCELONA,L-D 24H,2019-04-14 14:07:58.867659
5,REPSOL,1.409,1.539,1.349,"CALLE 6 SECTOR C ZONA FRANCA, S/N",BARCELONA,L-D 24H,2019-04-14 14:07:58.867659
6,BP,1.379,1.519,1.289,"CALLE JOAN GÜELL, 207",BARCELONA,L 06,2019-04-14 14:07:58.867659
7,GALP,1.414,1.554,1.294,"PASEO Pº VALLE DE HEBRÓN, 72",BARCELONA,L-D 24H,2019-04-14 14:07:58.867659
8,GALP,1.384,1.534,1.289,"CALLE TAJO, 28",BARCELONA,L-D 24H,2019-04-14 14:07:58.867659
9,CEPSA,1.409,1.524,1.319,"AVENIDA CLOT, 181",BARCELONA,L-D 06,2019-04-14 14:07:58.867659


In [22]:
# Exportamos los datos en un archivos csv
df.to_csv("Precio_combustible_Barcelona_provincia.csv", sep=',')